## HW 2: Yoda-speak 
### COSC 426: Fall 2025, Colgate University

Use this notebook to test your grammar, load in the results from your experiment, display them, and answer the question(s). Feel free to add as many code and markdown chunks as you would like in each of the sub-sections. 

**If you use any external resources (e.g., code snippets, reference articles), please cite them in comments or text!**

SAE: VP PP & NP PP
YS: PP VP & NP PP

## Part 1: Writing the grammar for `yoda-speak`

#### Testing sentences without Prepositional Phrases

Use the following code chunk to verify that your grammar correctly parses the sentences without prepositional phrases. Make sure that: 

1. If an NP is SubjNP in SAE, it is also SubjNP in `yoda-speak`
2. If an NP is ObjNP in SAE, it is also ObjNP in `yoda-speak`

In [44]:
import HW2

sents = [{'sae': 'you seek the Force', 'yoda': 'the Force you seek'},
         {'sae': 'the Jedi follow the path', 'yoda': 'the path the Jedi follow'},
         {'sae': 'I teach you', 'yoda': 'you I teach'},
         {'sae': 'the Jedi fight', 'yoda': 'fight the Jedi do'},
         {'sae': 'you sleep', 'yoda': 'sleep you do'}
        ]

for sent in sents: 
    print('=='*50)
    print('SAE')
    HW2.print_parses(sent['sae'], 'hw2_grammar_sae.txt',)
    print('--'*30)
    print('YODA')
    HW2.print_parses(sent['yoda'], 'hw2_grammar_yoda.txt')
    

SAE
             ROOT                
              |                   
              S                  
    __________|____               
   |               VP            
   |      _________|____          
 SubjNP  |            ObjNP      
   |     |              |         
   NP    |              NP       
   |     |          ____|_____    
Pronoun  TV       Det         N  
   |     |         |          |   
  you   seek      the       Force

------------------------------------------------------------
YODA
           ROOT             
            |                
            S               
       _____|____________    
    ObjNP        SubjNP  |  
      |            |     |   
      NP           NP    |  
  ____|_____       |     |   
Det         N   Pronoun  TV 
 |          |      |     |   
the       Force   you   seek

SAE
                 ROOT                
                  |                   
                  S                  
       ___________|_____             

#### Testing sentences with Prepositional Phrases

Use the following code chunk to verify that your grammar correctly parses the sentences with prepositional phrases. Make sure that: 

1. If an NP is SubjNP in SAE, it is also SubjNP in `yoda-speak`
2. If an NP is ObjNP in SAE, it is also ObjNP in `yoda-speak`
3. If a PP is the sister of the verb in SAE, it is also the sister of the verb in `yoda-speak`

In [45]:
sents = [{'sae': 'the Jedi fight in the night', 'yoda': 'in the night fight the Jedi do'},
         {'sae': 'the Jedi fight in the night in the rain', 'yoda': 'in the night in the rain fight the Jedi do'},
         {'sae': 'the Jedi fight in the night in the rain', 'yoda': 'in the rain in the night fight the Jedi do'},
         {'sae': 'the darkness in the night teach the Jedi', 'yoda': 'the Jedi the darkness in the night teach'},
         {'sae': 'the Jedi seek the Force through the darkness', 'yoda': 'the Force through the darkness the Jedi seek'}
        ]

for sent in sents: 
    print('=='*50)
    print('SAE')
    HW2.print_parses(sent['sae'], 'hw2_grammar_sae.txt',)
    print('--'*30)
    print('YODA')
    HW2.print_parses(sent['yoda'], 'hw2_grammar_yoda.txt')
    

SAE
                 ROOT                      
                  |                         
                  S                        
       ___________|________                 
      |                    VP              
      |            ________|___             
    SubjNP        |            PP          
      |           |     _______|___         
      NP          VP   |           NP      
  ____|_____      |    |        ___|____    
Det         N     IV   P      Det       N  
 |          |     |    |       |        |   
the        Jedi fight  in     the     night

------------------------------------------------------------
YODA
                   ROOT                    
                    |                       
                    S                      
          __________|____________________   
         VP                   |          | 
      ___|__________          |          |  
     PP             |       SubjNP       | 
  ___|___           |         |         

## Part 2: Evaluate whether `distilgpt` treats sentences from `yoda-speak` as being grammatical

In this section, include: 

* Code to create the evaluation data file (and a discussion if you choose to include additional sentences)
* The results from your experiment. This can be in the format of a dataframe with just the required rows, or as a plot.
* Discussion of the results

In [46]:
import pandas as pd
df = pd.read_csv('./results/results.tsv', delimiter='\t', index_col=0)
df

,model,acc,diff,expected,unexpected,macrodiff
0,distilbert/distilgpt2,0.0,-0.057432,0.217685,0.275117,-0.057432


Preferr yoda-speak over SAE, the model never did. In all 20 of the Yoda-SAE pairs tested, a higher probability than its yoda-speak counterpart, the SAE sentence was assigned. That the model considers yoda-speak ungrammatical, this strong bias suggests. An expected outcome, this is, given that on a dataset of SAE predominantly, trained the model likely was.

(I was having too much fun with it)

## Part 3: Finetune `distilgpt` on `yoda-speak` and evaluate it

In this section, include: 

* Code to create the finetuning data file. 
* The results from your experiment. This can be in the format of a dataframe with just the required rows, or as a plot.
* Discussion of the results

In [43]:
import HW2
import csv
grammar = HW2.create_pcfg('./hw2_grammar_yoda.txt')
data = HW2.generate_sentences(grammar, 10000)

with open('./data/train.tsv', 'w', newline='', encoding='utf-8') as tsvfile:
    tsv_writer = csv.writer(tsvfile, delimiter='\t')
    tsv_writer.writerow(['text'])
    tsv_writer.writerows([[r[0] ]for r in data[:9000]])

with open('./data/val.tsv', 'w', newline='', encoding='utf-8') as tsvfile:
    tsv_writer = csv.writer(tsvfile, delimiter='\t')
    tsv_writer.writerow(['text'])
    tsv_writer.writerows([[r[0] ]for r in data[9000:]])


## Part 4 (Optional): Train  `distilgpt` on `yoda-speak` from scratch and evaluate it.
In this section, include: 

* Code to create the training data file.
* The results from your experiment. This can be in the format of a dataframe with just the required rows, or as a plot.
* Discussion of the results and the difference between training and finetuning